Enabling Structured Data Analysis using Langchain SQL Agent integration with Amazon Bedrock 

In [2]:
%%writefile requirements.txt
boto3>=1.28.57
awscli>=1.29.57
botocore>=1.31.57
sqlalchemy==1.4.47
langchain==0.0.281
langchain_experimental
redshift-connector==2.0.910
sqlalchemy-redshift==0.8.14

Overwriting requirements.txt


In [3]:
!pip install -r requirements.txt

In [11]:
import json
import os
import sys

import boto3
from langchain.llms.bedrock import Bedrock

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

def get_bedrock_client():
    bedrock_client = boto3.client("bedrock-runtime", region_name=os.environ.get("AWS_DEFAULT_REGION", None))
    return bedrock_client

def create_bedrock_llm(bedrock_client, model_version_id):
    bedrock_llm = Bedrock(
        model_id=model_version_id, 
        client=bedrock_client,
        model_kwargs={'temperature': 0}
        )
    return bedrock_llm

In [12]:
from requests import get

ip = get('https://api.ipify.org').content.decode('utf8')
print('My public IP address is: {}'.format(ip))

My public IP address is: 52.27.224.182


In [13]:
from sqlalchemy import create_engine
from langchain import PromptTemplate,SagemakerEndpoint,SQLDatabase, LLMChain
from langchain_experimental.sql import SQLDatabaseChain
from sqlalchemy.engine.url import URL
import sqlalchemy as sa

# build the sqlalchemy URL
url = URL.create(
drivername='redshift+redshift_connector', # indicate redshift_connector driver and dialect will be used
host='<Redshift Host>', # Amazon Redshift host
port=5439, # Amazon Redshift port
database='<Database>', # Amazon Redshift database
username='<Username>', # Amazon Redshift username
password='<Password>' # Amazon Redshift password
)

engine_redshift = create_engine(url)
dbredshift = SQLDatabase(engine_redshift)

In [20]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.prompts import PromptTemplate
from langchain.agents.agent_types import AgentType

prompt_template = PromptTemplate(
input_variables=["nlq"],
template="\n\nHuman: \"{nlq} \"\n\nAssistant:",
)

bedrock_client = get_bedrock_client()
bedrock_llm = create_bedrock_llm(bedrock_client, "anthropic.claude-v1")
llm_chain = LLMChain(llm=bedrock_llm, prompt=prompt)
toolkit = SQLDatabaseToolkit(db=dbredshift, llm=bedrock_llm)

agent_executor = create_sql_agent(
    bedrock_llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

In [21]:
agent_executor.run(prompt_template.format(nlq="Describe the customer table"))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: category, customer, date, event, lineitem, listing, nation, orders, part, partsupp, region, sales, supplier, users, venue
Thought: The customer table seems most relevant to the question.
Action: sql_db_schema
Action Input: customer
Observation: 
CREATE TABLE customer (
	c_custkey BIGINT NOT NULL, 
	c_name VARCHAR(25), 
	c_address VARCHAR(40), 
	c_nationkey BIGINT, 
	c_phone VARCHAR(15), 
	c_acctbal NUMERIC(18, 4), 
	c_mktsegment VARCHAR(10), 
	c_comment VARCHAR(117)
) DISTSTYLE ALL

/*
3 rows from customer table:
c_custkey	c_name	c_address	c_nationkey	c_phone	c_acctbal	c_mktsegment	c_comment
1	Customer#000000001	IVhzIApeRb ot,c,E	15	25-989-741-2988	711.5600	BUILDING	to the even, regular platelets. regular, ironic epitaphs nag e
2	Customer#000000002	XSTf4,NCwDVaWNe6tEgvwfmRchLXak	13	23-768-687-3665	121.6500	AUTOMOBILE	l accounts. blithely ironic theodolites integrate boldly: caref
3	Customer#0

/opt/conda/lib/python3.10/site-packages/langchain/utilities/sql_database.py:351: SAWarning: Dialect redshift+redshift_connector does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  sample_rows_result = connection.execute(command)  # type: ignore


 I now know the final answer
Final Answer: The customer table contains information about customers, including their customer key, name, address, nation, phone number, account balance, market segment, and a comment.

> Finished chain.


'The customer table contains information about customers, including their customer key, name, address, nation, phone number, account balance, market segment, and a comment.'

In [22]:
agent_executor.run(prompt_template.format(nlq="Whats the number of customer"))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: category, customer, date, event, lineitem, listing, nation, orders, part, partsupp, region, sales, supplier, users, venue
Thought: The customer table likely has information on customers. I should query its schema.
Action: sql_db_schema
Action Input: customer
Observation: 
CREATE TABLE customer (
	c_custkey BIGINT NOT NULL, 
	c_name VARCHAR(25), 
	c_address VARCHAR(40), 
	c_nationkey BIGINT, 
	c_phone VARCHAR(15), 
	c_acctbal NUMERIC(18, 4), 
	c_mktsegment VARCHAR(10), 
	c_comment VARCHAR(117)
) DISTSTYLE ALL

/*
3 rows from customer table:
c_custkey	c_name	c_address	c_nationkey	c_phone	c_acctbal	c_mktsegment	c_comment
1	Customer#000000001	IVhzIApeRb ot,c,E	15	25-989-741-2988	711.5600	BUILDING	to the even, regular platelets. regular, ironic epitaphs nag e
2	Customer#000000002	XSTf4,NCwDVaWNe6tEgvwfmRchLXak	13	23-768-687-3665	121.6500	AUTOMOBILE	l accounts. blithely ironic theodolites integrate

'There are 15 million customers.'

In [23]:
agent_executor.run(prompt_template.format(nlq="List the distinct name of the nations"))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: category, customer, date, event, lineitem, listing, nation, orders, part, partsupp, region, sales, supplier, users, venue
Thought: The nation table seems most relevant for this question. I should query its schema.
Action: sql_db_schema
Action Input: nation
Observation: 
CREATE TABLE nation (
	n_nationkey BIGINT NOT NULL, 
	n_name VARCHAR(25), 
	n_regionkey BIGINT, 
	n_comment VARCHAR(152)
) DISTSTYLE ALL

/*
3 rows from nation table:
n_nationkey	n_name	n_regionkey	n_comment
0	ALGERIA	0	 haggle. carefully final deposits detect slyly agai
1	ARGENTINA	1	al foxes promise slyly according to the regular accounts. bold requests alon
2	BRAZIL	1	y alongside of the pending deposits. carefully special packages are about the ironic forges. slyly s
*/
Thought: I should query just the n_name column from the nation table.
Action: sql_db_query
Action Input: SELECT DISTINCT n_name FROM nation;
Observation: [(

'The distinct names of nations are:\nALGERIA\nARGENTINA\nBRAZIL\nCANADA\nEGYPT\nETHIOPIA\nFRANCE\nGERMANY'